<a href="https://colab.research.google.com/github/kin0969/ML-Homework/blob/master/Twitter_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive for read and write
from google.colab import drive
drive.mount('/content/gdrive')
training_label="/content/gdrive/My Drive/Colab Notebooks/ML Homework/RNN 2020/Raw Data/"+"training_label.txt"
training_nolabel="/content/gdrive/My Drive/Colab Notebooks/ML Homework/RNN 2020/Raw Data/"+"training_nolabel.txt"
testing_data="/content/gdrive/My Drive/Colab Notebooks/ML Homework/RNN 2020/Raw Data/"+"testing_data.txt"

predict="/content/gdrive/My Drive/Colab Notebooks/ML Homework/RNN 2020/Result/"+"predict.csv"
general_result_path="/content/gdrive/My Drive/Colab Notebooks/ML Homework/RNN 2020/Result/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


###Utils

In [0]:
# Utils.py
# General library
 
 import torch
 import numpy as np
 import pandas as pd
 import torch.optim as optim
 import torch.nn.functional as F

 def load_training_data(path,with_label):   
   # 把 training 時需要的 data 讀進來
   # 如果是 'training_label.txt'，需要讀取 label，如果是 'training_nolabel.txt'，不需要讀取 label
   if with_label=='Y':
     with open(path,'r') as f:
       lines=f.readlines()       
       lines=[line.strip('\n').split(' ') for line in lines]
     
     x= [line[2:] for line in lines]
     y= [line[0] for line in lines] 
     return x,y
   else:
     with open(path, 'r') as f:
            lines = f.readlines()
            x = [line.strip('\n').split(' ') for line in lines]
     return x


def load_testing_data(path):
    # 把 testing 時需要的 data 讀進來
    with open(path, 'r') as f:
        lines = f.readlines()
        
        # 先不要header, 再去分行符號，再用逗號分隔
        lines_staging=[line.strip('\n').split(",") for line in lines[1:]]
        # 再把整句Raw句子拿出來 
        lines_staging= [line[1:] for line in lines_staging]
        # 再使用句子中的空格split出得出每個字，line係object冇split, 所以加D野上去變成String
        lines_staging= ["".join(line).split(' ') for line in lines_staging]    

    return lines_staging

def acc_evaluation(outputs, labels):
  outputs[outputs>=0.5]=1
  outputs[outputs<0.5]=0
  correct=torch.sum(torch.eq(outputs,labels)).item()
  return correct

       
# #  x,y=load_training_data(training_label,'Y')
#  x=load_testing_data(testing_data)
#  print ('Testing Data',x[0:10],'\r\n')

#  x,y=load_training_data(training_label,'Y')
#  print ('Training',x[0:10])




### Train Word to Vector

In [8]:
# 這個 block 是用來訓練 word to vector 的 word embedding
# 注意！這個 block 在訓練 word to vector 時是用 cpu，可能要花到 10 分鐘以上

import os
import numpy as np
import pandas as pd
import argparse
from gensim.models import word2vec

def train_word2vec(x):
  model=word2vec.Word2Vec(x,size=250,window=5,min_count=5,workers=12,iter=10,sg=1)
  return model


if __name__=="__main__":
  print ("loading training data")
  train_x,y=load_training_data(training_label,'Y')
  train_x_no_label=load_training_data(training_nolabel,'N')

  print("loading testing data")
  test_x=load_testing_data(testing_data)

  model =train_word2vec(train_x + test_x)

  print ("Saving models")
  model.save(general_result_path+"w2v_all.model")

loading training data
loading testing data
Saving models


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
